In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import csv
import math
import pickle
import random

# **Read Inputs**

In [9]:
# Read user data
words = []
with open('C:/Users/Brandon/Documents/CSE323/HW2/Data/data.txt', 'r') as input:
    """
    TODO: Parse user input data to a list of touch point collections/lists 
    """
    Lines = input.readlines()
    for line in Lines:
        if line[0]=="=":
            words.append([])
        else:
            words[-1].append(line.split())
words

[[['i', '25.258852', '7.752883'], ['f', '13.50101', '11.854456']],
 [['a', '4.9218874', '13.221646'], ['t', '16.132853', '7.0692873']],
 [['f', '13.671909', '12.948208'],
  ['i', '25.942448', '7.70731'],
  ['r', '12.03128', '7.160434'],
  ['s', '7.48537', '13.540659'],
  ['t', '15.346718', '7.3427258']],
 [['y', '20.91802', '6.65913'],
  ['o', '29.633863', '8.071894'],
  ['u', '23.754942', '8.61877']],
 [['f', '15.244179', '12.67477'],
  ['a', '4.648449', '13.084928'],
  ['i', '25.703188', '7.4794445'],
  ['l', '31.103592', '13.221646']],
 [['w', '5.6396623', '8.482051'], ['e', '8.784202', '7.6617365']],
 [['r', '12.03128', '7.6161637'],
  ['u', '24.711975', '7.7984557'],
  ['n', '24.575256', '18.189108']],
 [['t', '15.551796', '8.390905'],
  ['h', '21.054739', '13.768523'],
  ['e', '9.262718', '8.254187']],
 [['r', '13.159212', '8.61877'],
  ['i', '26.831121', '7.3427258'],
  ['s', '7.3144712', '12.993782'],
  ['k', '27.719795', '12.629196']],
 [['o', '29.97566', '6.385692'], ['f', '1

In [3]:
# Read dictionary
with open('C:/Users/Brandon/Documents/CSE323/HW2/Data/unigram.dict', 'rb') as unigramModelFile:
    unigramModel = pickle.load(unigramModelFile)
unigramModelFile.close()

# Read keyboard data
keyboard_raw = pd.read_csv("C:/Users/Brandon/Documents/CSE323/HW2/Data/keyboard.csv")
keyboard = keyboard_raw[['key', 'x_mm', 'y_mm']]
keyboard

,key,x_mm,y_mm
0,a,4.025010,9.625024
1,b,18.900047,13.650034
2,c,12.950032,13.650034
3,d,9.975025,9.625024
4,e,8.487521,5.600014
5,f,12.950032,9.625024
6,g,15.925039,9.625024
7,h,18.900047,9.625024
8,i,23.362558,5.600014
9,j,21.875053,9.625024


# **Unigram Language Model Decoder**

In [4]:
# Keyboard size and dual Gaussian model parameters
key_width = 3
key_height = 4
a = 2.403
b = 0.017
c = 2.295
d = 0.016

def get_likelihood(p, mu, sigma):
    """
    TODO: Calculate the likelihood that a touch point p is from the 2D Gaussian distribution N(mu, sigma)
    """
    
    lik = (1/((2*math.pi*sigma[0])**.5)) * math.exp(-1/(2*sigma[0])*(float(p[1])-mu[0])**2) * (1/((2*math.pi*sigma[1])**.5)) * math.exp(-1/(2*sigma[1])*(float(p[2])-mu[1])**2)
    return lik

def is_letter(p, letter):
    """
    TODO: Determine if touch point p is located inside the boundary of the key: letter
    """
    x=keyboard.loc[keyboard['key']==letter].reset_index().x_mm[0]
    y=keyboard.loc[keyboard['key']==letter].reset_index().y_mm[0]
    if float(p[1])<x or float(p[1])>x+key_width:
        return False
    if float(p[2])<y or float(p[2])>y+key_height:
        return False
    return True 

def get_literal_string(touchpoints):
    """ 
    TODO: Compute the literal string using is_letter(p, letter) method for a collection of touch points that represents a word. 
          If a touch point does not fall inside any key boundary, use '?' to represent the corresponding character.
    """
    literal_string = ""
    for touchpoint in touchpoints:
        for letter in keyboard['key'].tolist():
            if is_letter(touchpoint, letter):
                literal_string = literal_string + letter
                break
        else:
            literal_string = literal_string + "?"
    return literal_string
def get_near_letters(touchpoint):
    nearletters = []
    for letter in keyboard['key'].tolist():
        if is_near_letter(touchpoint, letter, .5):
            nearletters.append(letter)
    if not nearletters:
        for letter in keyboard['key'].tolist():
            if is_near_letter(touchpoint, letter, .75):
                nearletters.append(letter)
    return nearletters
def is_near_letter(p, letter, error):
    x=keyboard.loc[keyboard['key']==letter].reset_index().x_mm[0]
    y=keyboard.loc[keyboard['key']==letter].reset_index().y_mm[0]
    if float(p[1]) < x - error * key_width or float(p[1]) > x + (1 + error) * key_width:
        return False
    if float(p[2]) < y - error * key_height or float(p[2]) > y + (1 + error) * key_height:
        return False
    return True

In [11]:
def unigram_lm_decoder(touchpoints):
    """
    A language decoder that uses the dual Gaussian touch point spatial disrtibution model and a unigram language model.
    Input: a list/collection of touch points that represents a certain word
    Output: the decoded word for the input
    
    TODO: Step a --- Get all possible words and their corresponding probabilities from the dictionary. 
          Use the length of the correct word to filter possible words
          You may also use the first and/or the last touchpoint to further narrow down possible words
    """
    nearlettersfirst = get_near_letters(touchpoints[0])
    nearletterslast = get_near_letters(touchpoints[len(touchpoints)-1])
    
    
    possible_words = {word : unigramModel[word] for word in unigramModel if len(word)==len(touchpoints) and word[0] in nearlettersfirst and word[len(word)-1] in nearletterslast}
    
    # Calculate p(w|s_1, s_2, ... s_n) ~ p(s_1, s_2, ..., s_n|w)*p(w) = \Pi(p(s_i|c_i))p(w) for each possible word
    p_w_s = dict()                  # Holds p(w|s_1, s_2, ..., s_n) for all possible words
    for item in possible_words:
        word =  item                  # TODO: The current possible word
        p_w =   possible_words[item]                  # TODO: Probability of the current possible word in the unigram language model
        p_s_w = 1                 # Holds p(s_1, s_2, ..., s_n|w) for the current possible word

        for j, letter in enumerate(list(word)):
            # TODO: Step b --- Apply the spatial model to get p(s_i|c_i)
            mu = [keyboard.loc[keyboard['key']==letter]['x_mm'].tolist()[0]+key_width/2, keyboard.loc[keyboard['key']==letter]['y_mm'].tolist()[0]+key_height/2]
            sigma = [a + b * key_width**2, c + d * key_height**2]
            p_s_c = get_likelihood(touchpoints[j], mu, sigma)
            # TODO: Step b --- Multiply the current p(s_i|c_i) to p(s_1, s_2, ..., s_n|W)
            p_s_w = p_s_w * p_s_c
        # TODO: Step c --- Calculate p(w|s_1, s_2, ... s_n) from p(s_1, s_2, ..., s_n|w) and p(w). Append the result to list
        p_w_s[item] = p_s_w * p_w
    # TODO: Step d ---- Choose word by the maximum of p(w|s_1, s_2, ..., s_n)
    
    decoded_word = max(p_w_s, key=p_w_s.get)
    
    return decoded_word

In [12]:
decoded_success_count = 0
literal_success_count = 0
decoded_words = []
literal_strings = []
correct_words = []
count=0
for touchpoints in words:
    """
    TODO: Use above methods to compute the correct word, decoded word, and the literal string for each touch point collection
          Append results to the corresponding list
          Update the decoded words/literal strings success count. 
              --- If the decoded word/literal string is the same as correct word, increase 1 to decoded words/literal strings success count
    """
    correct_word = ""
    for touchpoint in touchpoints:
        correct_word = correct_word + touchpoint[0]
    correct_words.append(correct_word)
    literal_strings.append(get_literal_string(touchpoints))
    decoded_words.append(unigram_lm_decoder(touchpoints))
    count+=1
    print(str(count) + "/" + str(len(words)))
# TODO: calculate the success rate for both the decoded words and the literal strings using the docoded word/literal string success count
for i in range(len(correct_words)):
    if decoded_words[i]==correct_words[i]:
        decoded_success_count += 1
    if literal_strings[i]==correct_words[i]:
        literal_success_count += 1
decoded_success = decoded_success_count / len(correct_words)
literal_success = literal_success_count / len(correct_words)


1/153
2/153
3/153
4/153
5/153
6/153
7/153
8/153
9/153
10/153
11/153
12/153
13/153
14/153
15/153
16/153
17/153
18/153
19/153
20/153
21/153
22/153
23/153
24/153
25/153
26/153
27/153
28/153
29/153
30/153
31/153
32/153
33/153
34/153
35/153
36/153
37/153
38/153
39/153
40/153
41/153
42/153
43/153
44/153
45/153
46/153
47/153
48/153
49/153
50/153
51/153
52/153
53/153
54/153
55/153
56/153
57/153
58/153
59/153
60/153
61/153
62/153
63/153
64/153
65/153
66/153
67/153
68/153
69/153
70/153
71/153
72/153
73/153
74/153
75/153
76/153
77/153
78/153
79/153
80/153
81/153
82/153
83/153
84/153
85/153
86/153
87/153
88/153
89/153
90/153
91/153
92/153
93/153
94/153
95/153
96/153
97/153
98/153
99/153
100/153
101/153
102/153
103/153
104/153
105/153
106/153
107/153
108/153
109/153
110/153
111/153
112/153
113/153
114/153
115/153
116/153
117/153
118/153
119/153
120/153
121/153
122/153
123/153
124/153
125/153
126/153
127/153
128/153
129/153
130/153
131/153
132/153
133/153
134/153
135/153
136/153
137/153
138/153
139/

In [7]:
# TODO: Write to results.txt
with open("C:/Users/Brandon/Documents/CSE323/HW2/Data/results.txt", 'w') as output:
    # The first line: success_rate(decoded_words), success_rate(literal_strings)
    output.write("" + str(decoded_success) + ", " + str(literal_success) + "\n")
    for i in range(len(correct_words)):
        # Each line after: correct_word, decoded_word, literal_string
        output.write("" + str(correct_words[i]) + ", " + str(decoded_words[i]) + ", " + str(literal_strings[i]) + "\n")
output.close()